## Deliverable 2. Create a Customer Travel Destinations Map.

# Import Dependencies and Files

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.82,73,100,18.32,light rain
1,1,Port Shepstone,ZA,-30.7414,30.4550,71.74,72,100,5.12,overcast clouds
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,74.26,74,18,19.24,few clouds
3,3,Comodoro Rivadavia,AR,-45.8667,-67.5000,53.60,87,75,2.30,light intensity drizzle
4,4,Talara,PE,-4.5772,-81.2719,71.46,77,26,12.62,scattered clouds


# Filter the DataFrame

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria as a float variable
min_temp = float(input("Enter the minimum desired tempature for your vacation location: "))
max_temp = float(input("Enter the maximum desired tempature for your vacation location: "))

Enter the minimum desired tempature for your vacation location: 60
Enter the maximum desired tempature for your vacation location: 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp']<=max_temp)]
filtered_city_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.82,73,100,18.32,light rain
1,1,Port Shepstone,ZA,-30.7414,30.4550,71.74,72,100,5.12,overcast clouds
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,74.26,74,18,19.24,few clouds
4,4,Talara,PE,-4.5772,-81.2719,71.46,77,26,12.62,scattered clouds
6,6,Olinda,BR,-8.0089,-34.8553,78.80,78,40,4.61,scattered clouds
7,7,Palwal,IN,28.1500,77.3333,87.80,19,0,9.22,dust
9,9,Victoria,HK,22.2855,114.1577,82.99,77,73,1.01,broken clouds
13,13,East London,ZA,-33.0153,27.9116,64.40,88,20,4.61,few clouds
15,15,Flinders,AU,-34.5833,150.8552,75.00,68,0,5.01,clear sky
19,19,Tarakan,ID,3.3000,117.6333,82.29,74,74,4.41,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
filtered_city_df.count()

City_ID                343
City                   343
Country                338
Lat                    343
Lng                    343
Max Temp               343
Humidity               343
Cloudiness             343
Wind Speed             343
Current Description    343
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_city_df = filtered_city_df.dropna()
# Check that na rows were dropped by recounting/checking for empty rows
clean_city_df.count()

City_ID                338
City                   338
Country                338
Lat                    338
Lng                    338
Max Temp               338
Humidity               338
Cloudiness             338
Wind Speed             338
Current Description    338
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.82,light rain,-23.1203,-134.9692,
1,Port Shepstone,ZA,71.74,overcast clouds,-30.7414,30.4550,
2,Arraial Do Cabo,BR,74.26,few clouds,-22.9661,-42.0278,
4,Talara,PE,71.46,scattered clouds,-4.5772,-81.2719,
6,Olinda,BR,78.80,scattered clouds,-8.0089,-34.8553,
7,Palwal,IN,87.80,dust,28.1500,77.3333,
9,Victoria,HK,82.99,broken clouds,22.2855,114.1577,
13,East London,ZA,64.40,few clouds,-33.0153,27.9116,
15,Flinders,AU,75.00,clear sky,-34.5833,150.8552,
19,Tarakan,ID,82.29,broken clouds,3.3000,117.6333,


# API Call

In [8]:
# Use Google NearBySearch API to find hotels near by our location
# https://developers.google.com/maps/documentation/places/web-service/search?hl=en_US

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{long}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: # place the hotel results name in to our dataframe if it finds one
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): # except when it does not, print that the hotel not found 
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


# Clean and Export the DataFrame

In [9]:
# 7. Drop the rows where there is no Hotel Name.
# https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe

# First replace the blanks with Nan so that we can delete those rows 
hotel_df['Hotel Name'].replace("",np.nan,inplace=True)
#hotel_df

# Then we can use .dropna() to drop rows with na in it 
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.82,light rain,-23.1203,-134.9692,Pension Maro'i
1,Port Shepstone,ZA,71.74,overcast clouds,-30.7414,30.4550,My Den Beachfront Bed and Breakfast | Self-Cat...
2,Arraial Do Cabo,BR,74.26,few clouds,-22.9661,-42.0278,Pousada Porto Praia
4,Talara,PE,71.46,scattered clouds,-4.5772,-81.2719,Hostal Oro Negro
6,Olinda,BR,78.80,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
...,...,...,...,...,...,...,...
687,Yenagoa,NG,78.39,broken clouds,4.9247,6.2642,Aridolf Resort Wellness and Spa
689,Senanga,ZM,67.48,few clouds,-16.1167,23.2667,Christopher Nasilele House
690,Omboue,GA,73.69,overcast clouds,-1.5746,9.2618,Hotel Olako
699,Lakatoro,VU,81.84,overcast clouds,-16.0999,167.4164,Lakatoro Palm Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

# Create the Google Map Visual

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# This will display an info box when we click on the marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.

# Define the center and zoom level for the figure
fig = gmaps.figure(center=(10.0, 5.0), zoom_level=1.5)
# Create a layer on the figure/map that marks each location and displays the info box 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the created layer to the figure
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))